# Generative networks

Recurrent Neural Networks (RNNs) and their gated cell variants such as Long Short Term Memory Cells (LSTMs) and Gated Recurrent Units (GRUs) provided a mechanism for language modeling, i.e. they can learn word ordering and provide predictions for next word in a sequence. This allows us to use RNNs for **generative tasks**, such as ordinary text generation, machine translation, and even image captioning.

In RNN architecture we discussed in the previous unit, each RNN unit produced next next hidden state as an output. However, we can also add another output to each recurrent unit, which would allow us to output a **sequence** (which is equal in length to the original sequence). Moreover, we can use RNN units that do not accept an input at each step, and just take some initial state vector, and then produce a sequence of outputs.

This allows for different neural architectures that are shown in the picture below:

循环神经网络 (RNN) 及其门控单元变体，例如长短期记忆单元 (LSTM) 和门控循环单元 (GRU)，提供了一种语言建模机制，即它们可以学习词序并预测序列中的下一个词 . 这使我们能够将 RNN 用于**生成任务**，例如普通文本生成、机器翻译，甚至图像字幕。

在我们在上一单元讨论的 RNN 架构中，每个 RNN 单元产生下一个隐藏状态作为输出。 然而，我们也可以向每个循环单元添加另一个输出，这将允许我们输出一个**序列**（其长度等于原始序列）。 此外，我们可以使用在每一步不接受输入的 RNN 单元，只采用一些初始状态向量，然后产生一系列输出。

这允许不同的神经架构，如下图所示：

<img alt="Image showing common recurrent neural network patterns." src="images/6-generative-networks-1.jpg" align="middle" />
*Image from blog post "Unreasonable Effectiveness of Recurrent Neural Networks" by Andrej Karpaty*

* **One-to-one** is a traditional neural network with one input and one output
* **One-to-many** is a generative architecture that accepts one input value, and generates a sequence of output values. For example, if we want to train `image captioning` network that would produce a textual description of a picture, we can have a picture as input, pass it through CNN to obtain hidden state, and then have recurrent chain generate caption word-by-word
* **Many-to-one** corresponds to RNN architectures we described in the previous unit, such as `text classification`
* **Many-to-many**, or **sequence-to-sequence** corresponds to tasks such as `machine translation` or `language translation`, where we have first RNN collect all information from the input sequence into the hidden state, and another RNN chain unrolls this state into the output sequence.

In this unit, we will focus on simple *generative models* that help us generate text. For simplicity, let's build a **character-level network**, which generates news articles text letter by letter. While generating news articles may seem quite impractical, the same idea of sequence generation is used in many practical tasks, such as machine translation, text summarization, etc.

During training, we need to take some text corpus, and split it into letter sequences. 

*图片来自 Andrej Karpaty 的博客文章“递归神经网络的不合理有效性”*

* **一对一**是传统的神经网络，一个输入一个输出
* **一对多** 是一种生成架构，它接受一个输入值，并生成一系列输出值。 例如，如果我们想训练生成图片文本描述的“图像字幕”网络，我们可以将图片作为输入，将其传递给 CNN 以获得隐藏状态，然后让循环链逐字生成字幕 -单词
* **多对一**对应于我们在上一单元中描述的 RNN 架构，例如“文本分类”
* **多对多**，或 **sequence-to-sequence** 对应于诸如“机器翻译”或“语言翻译”之类的任务，我们首先让 RNN 将输入序列中的所有信息收集到 隐藏状态，另一个 RNN 链将此状态展开到输出序列中。

在本单元中，我们将专注于帮助我们生成文本的简单*生成模型*。 为简单起见，让我们构建一个**字符级网络**，逐个字母地生成新闻文章文本。 虽然生成新闻文章可能看起来很不切实际，但在许多实际任务中都使用了相同的序列生成思想，例如机器翻译、文本摘要等。

在训练过程中，我们需要获取一些文本语料库，并将其拆分为字母序列。

In [3]:
import torch
import torchtext
import numpy as np
from torchnlp import *
train_dataset,test_dataset,classes,vocab = load_dataset()

Loading dataset...
Building vocab...


## Building character vocabulary

To build character-level generative network, we need to split text into individual characters instead of words. This can be done by defining a different tokenizer.  For example:
- `vocab.stoi` - maps token strings in the vacab to a numeric identifier
- `vocab.itos` - maps a number index from the vocab to a string 

要构建字符级生成网络，我们需要将文本拆分为单个字符而不是单词。 这可以通过定义不同的分词器来完成。 例如：
- `vocab.stoi` - 将 vacab 中的标记字符串映射到数字标识符
- `vocab.itos` - 将数字索引从词汇表映射到字符串

In [4]:
def char_tokenizer(words):
    return list(words) #[word for word in words]

counter = collections.Counter()
for (label, line) in train_dataset:
    counter.update(char_tokenizer(line))
vocab = torchtext.vocab.Vocab(counter)

vocab_size = len(vocab)
print(f"Vocabulary size = {vocab_size}")
print(f"Encoding of 'a' is {vocab.stoi['a']}")
print(f"Character with code 13 is {vocab.itos[13]}")

Vocabulary size = 84
Encoding of 'a' is 4
Character with code 13 is h


Let's see the example of how we can encode the text from our dataset:

让我们看看如何对数据集中的文本进行编码的示例：

In [9]:
def enc(x):
    return torch.LongTensor(encode(x,voc=vocab,tokenizer=char_tokenizer))

print(f'sample text:\n{train_dataset[0][1]} ')
print(f'\nCharater encoding:\n{enc(train_dataset[0][1])} ')

sample text:
Wall St. Bears Claw Back Into the Black (Reuters) Reuters - Short-sellers, Wall Street's dwindling\band of ultra-cynics, are seeing green again. 

Charater encoding:
tensor([43,  4, 11, 11,  2, 26,  5, 23,  2, 38,  3,  4, 10,  9,  2, 31, 11,  4,
        21,  2, 38,  4, 14, 25,  2, 34,  8,  5,  6,  2,  5, 13,  3,  2, 38, 11,
         4, 14, 25,  2, 55, 37,  3, 15,  5,  3, 10,  9, 56,  2, 37,  3, 15,  5,
         3, 10,  9,  2, 29,  2, 26, 13,  6, 10,  5, 29,  9,  3, 11, 11,  3, 10,
         9, 27,  2, 43,  4, 11, 11,  2, 26,  5, 10,  3,  3,  5, 58,  9,  2, 12,
        21,  7,  8, 12, 11,  7,  8, 18, 61, 22,  4,  8, 12,  2,  6, 19,  2, 15,
        11,  5, 10,  4, 29, 14, 20,  8,  7, 14,  9, 27,  2,  4, 10,  3,  2,  9,
         3,  3,  7,  8, 18,  2, 18, 10,  3,  3,  8,  2,  4, 18,  4,  7,  8, 23]) 


## Training a generative RNN

The way we will train RNN to generate text is the following. On each step, we will take a sequence of characters of length `nchars`, and ask the network to generate the next output character for each input character:

我们将训练 RNN 生成文本的方式如下。 在每一步中，我们将采用长度为 nchars 的字符序列，并要求网络为每个输入字符生成下一个输出字符：

<img alt="Image showing an example RNN generation of the word 'HELLO'." src="images/6-generative-networks-2.png" align="middle" />

Depending on the actual scenario, we may also want to include some special characters, such as *end-of-sequence* `<eos>`. In our case, we just want to train the network for endless text generation, thus we will fix the size of each sequence to be equal to `nchars` tokens. Consequently, each training example will consist of `nchars` inputs and `nchars` outputs (which are input sequence shifted one symbol to the left). Minibatch will consist of several such sequences.

The way we will generate minibatches is to take each news text of length `l`, and generate all possible input-output combinations from it (there will be `l-nchars` such combinations). They will form one minibatch, and size of minibatches would be different at each training step. 

根据实际场景，我们可能还希望包含一些特殊字符，例如 *end-of-sequence* `<eos>`。 在我们的例子中，我们只想训练网络生成无穷无尽的文本，因此我们将每个序列的大小固定为等于 `nchars` 标记。 因此，每个训练示例都将包含`nchars`输入和`nchars`输出（输入序列向左移动一个符号）。 小批量将由几个这样的序列组成。

我们生成小批量的方法是获取每个长度为`l`的新闻文本，并从中生成所有可能的输入输出组合（将有`l-nchars`个这样的组合）。 它们将形成一个小批量，并且在每个训练步骤中小批量的大小都会不同。

In [10]:
nchars = 100

def get_batch(s,nchars=nchars):
    ins = torch.zeros(len(s)-nchars,nchars,dtype=torch.long,device=device)
    outs = torch.zeros(len(s)-nchars,nchars,dtype=torch.long,device=device)
    for i in range(len(s)-nchars):
        ins[i] = enc(s[i:i+nchars])
        outs[i] = enc(s[i+1:i+nchars+1])
    return ins,outs

get_batch(train_dataset[0][1])

(tensor([[43,  4, 11,  ..., 18, 61, 22],
         [ 4, 11, 11,  ..., 61, 22,  4],
         [11, 11,  2,  ..., 22,  4,  8],
         ...,
         [37,  3, 15,  ...,  4, 18,  4],
         [ 3, 15,  5,  ..., 18,  4,  7],
         [15,  5,  3,  ...,  4,  7,  8]]),
 tensor([[ 4, 11, 11,  ..., 61, 22,  4],
         [11, 11,  2,  ..., 22,  4,  8],
         [11,  2, 26,  ...,  4,  8, 12],
         ...,
         [ 3, 15,  5,  ..., 18,  4,  7],
         [15,  5,  3,  ...,  4,  7,  8],
         [ 5,  3, 10,  ...,  7,  8, 23]]))

Now let's define the generative network. It can be based on any recurrent cell which we discussed in the previous unit (simple, LSTM or GRU). In our example we will use LSTM.

Because the network takes characters as input, and vocabulary size is pretty small, we do not need embedding layer, one-hot-encoded input can directly go to LSTM cell. However, because we pass character numbers as input, we need to one-hot-encode them before passing to LSTM. This is done by calling `one_hot` function during `forward` pass. Output encoder would be a linear layer that will convert hidden state into one-hot-encoded output.

现在让我们定义生成网络。 它可以基于我们在上一单元中讨论过的任何循环单元（简单的、LSTM 或 GRU）。 在我们的示例中，我们将使用 LSTM。

因为网络以字符作为输入，词汇量很小，我们不需要嵌入层，one-hot-encoded 输入可以直接进入 LSTM 单元。 但是，因为我们将字符数字作为输入传递，所以我们需要在传递给 LSTM 之前对它们进行单热编码。 这是通过在`前向`传递期间调用`one_hot`函数来完成的。 输出编码器将是一个线性层，它将隐藏状态转换为单热编码输出。

In [11]:
class LSTMGenerator(torch.nn.Module):
    def __init__(self, vocab_size, hidden_dim):
        super().__init__()
        self.rnn = torch.nn.LSTM(vocab_size,hidden_dim,batch_first=True)
        self.fc = torch.nn.Linear(hidden_dim, vocab_size)

    def forward(self, x, s=None):
        x = torch.nn.functional.one_hot(x,vocab_size).to(torch.float32)
        x,s = self.rnn(x,s)
        return self.fc(x),s

During training, we want to be able to sample generated text. To do that, we will define a `generate` function that will produce an output string of length `size`, starting from the initial string `start`.

The way it works is the following:
- First, we will pass the whole start string through the network, and take output state `s`
and next predicted character `out`. 
- Since `out` is one-hot encoded, we take `argmax` to get the index of the character `nc` in the vocabulary, and use `itos` to figure out the actual character and append it to the resulting list of characters `chars`. 
- This process of generating one character is repeated `size` times to generate required number of characters.  

在训练期间，我们希望能够对生成的文本进行采样。 为此，我们将定义一个 `generate` 函数，该函数将从初始字符串 `start` 开始生成长度为 `size` 的输出字符串。

它的工作方式如下：
- 首先，我们将通过网络传递整个起始字符串，并获取输出状态 `s`
和下一个预测字符`out`。
- 由于 `out` 是单热编码的，我们使用 `argmax` 来获取词汇表中字符 `nc` 的索引，并使用 `itos` 找出实际字符并将其附加到结果字符列表中 `字符`。
- 这个生成一个字符的过程重复 `size` 次以生成所需数量的字符。

In [12]:
def generate(net,size=100,start='today '):
        chars = list(start)
        out, s = net(enc(chars).view(1,-1).to(device))
        for i in range(size):
            nc = torch.argmax(out[0][-1])
            chars.append(vocab.itos[nc])
            out, s = net(nc.view(1,-1),s)
        return ''.join(chars)

Now let's do the training! The training loop is almost the same as in all our previous examples, but instead of accuracy we print sampled generated text every 1000 epochs.

Special attention needs to be paid to the way we compute loss. We need to compute loss given one-hot-encoded output `out`, and expected text `text_out`, which is the list of character indices. Luckily, the `cross_entropy` function expects unnormalized network output as first argument, and class number as the second, which is exactly what we have. It also performs automatic averaging over minibatch size.

We also limit the training by `samples_to_train` samples, in order not to wait for too long. We encourage you to experiment and try longer training, possibly for several epochs (in which case you would need to create another loop around this code).

现在让我们开始训练吧！ 训练循环与我们之前的所有示例几乎相同，但是我们每 1000 个 epoch 打印一次生成的文本样本而不是准确性。

需要特别注意我们计算损失的方式。 我们需要计算给定单热编码输出` out` 和预期文本 `text_out`（字符索引列表）的损失。 幸运的是，`cross_entropy` 函数期望非标准化网络输出作为第一个参数，类号作为第二个参数，这正是我们所拥有的。 它还对小批量大小执行自动平均。

我们还通过 `samples_to_train` 样本限制训练，以免等待太久。 我们鼓励您试验并尝试更长时间的训练，可能持续几个时期（在这种情况下，您需要围绕此代码创建另一个循环）。

In [13]:
net = LSTMGenerator(vocab_size,64).to(device)

samples_to_train = 10000
optimizer = torch.optim.Adam(net.parameters(),0.01)
loss_fn = torch.nn.CrossEntropyLoss()
net.train()
for i,x in enumerate(train_dataset):
    # x[0] is class label, x[1] is text
    if len(x[1])-nchars<10:
        continue
    samples_to_train-=1
    if not samples_to_train: break
    text_in, text_out = get_batch(x[1])
    optimizer.zero_grad()
    out,s = net(text_in)
    loss = torch.nn.functional.cross_entropy(out.view(-1,vocab_size),text_out.flatten()) #cross_entropy(out,labels)
    loss.backward()
    optimizer.step()
    if i%1000==0:
        print(f"Current loss = {loss.item()}")
        print(generate(net))

Current loss = 4.4620537757873535
today hhllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllll
Current loss = 2.118957281112671
today and and and and and and and and and and and and and and and and and and and and and and and and and 
Current loss = 1.6416268348693848
today and the company to a the company to a the company to a the company to a the company to a the company
Current loss = 2.464548110961914
today and a the US the product to the US the product to the US the product to the US the product to the US
Current loss = 1.7117735147476196
today and the state the state the state the state the state the state the state the state the state the st
Current loss = 1.667232632637024
today a second the company and the company and the company and the company and the company and the company
Current loss = 1.9207301139831543
today a first the first the first the first the first the first the first the first the first the first th
Current loss = 1

This example already generates some pretty good text, but it can be further improved in several ways:
* **Better minibatch generation**. The way we prepared data for training was to generate one minibatch from one sample. This is not ideal, because minibatches are all of different sizes, and some of them even cannot be generated, because the text is smaller than `nchars`. Also, small minibatches do not load GPU sufficiently enough. It would be wiser to get one large chunk of text from all samples, then generate all input-output pairs, shuffle them, and generate minibatches of equal size.
* **Multilayer LSTM**. It makes sense to try 2 or 3 layers of LSTM cells. As we mentioned in the previous unit, each layer of LSTM extracts certain patterns from text, and in case of character-level generator we can expect lower LSTM level to be responsible for extracting syllables, and higher levels - for words and word combinations. This can be simply implemented by passing number-of-layers parameter to LSTM constructor.
* You may also want to experiment with **GRU units** and see which ones perform better, and with **different hidden layer sizes**. Too large hidden layer may result in overfitting (e.g. network will learn exact text), and smaller size might not produce good result.

这个例子已经生成了一些非常好的文本，但它可以通过多种方式进一步改进：
* **更好的小批量生成**。 我们为训练准备数据的方式是从一个样本生成一个小批量。 这并不理想，因为 minibatch 的大小各不相同，有些甚至无法生成，因为文本小于 `nchars`。 此外，小的 minibatches 无法充分加载 GPU。 明智的做法是从所有样本中获取一大块文本，然后生成所有输入-输出对，打乱它们，并生成大小相等的小批量。
* **多层 LSTM**。 尝试 2 层或 3 层 LSTM 单元是有意义的。 正如我们在上一单元中提到的，LSTM 的每一层都从文本中提取某些模式，在字符级生成器的情况下，我们可以预期较低的 LSTM 层负责提取音节，而较高的层负责提取单词和单词组合。 这可以通过将层数参数传递给 LSTM 构造函数来简单地实现。
* 您可能还想尝试使用 **GRU 单元** 并查看哪些表现更好，并使用 **不同的隐藏层大小**。 太大的隐藏层可能会导致过度拟合（例如网络将学习准确的文本），而较小的尺寸可能不会产生好的结果。

## Soft text generation and temperature

In the previous definition of `generate`, we were always taking the character with highest probability as the next character in generated text. This resulted in the fact that the text often "cycled" between the same character sequences again and again, like in this example:

在之前的 `generate` 定义中，我们总是将概率最高的字符作为生成文本中的下一个字符。 这导致文本经常在相同的字符序列之间一次又一次地“循环”，就像这个例子：

```
today of the second the company and a second the company ...
```

However, if we look at the probability distribution for the next character, it could be that the difference between a few highest probabilities is not huge, e.g. one character can have probability 0.2, another - 0.19, etc. For example, when looking for the next character in the sequence '*play*', next character can equally well be either space, or **e** (as in the word *player*).

This leads us to the conclusion that it is not always "fair" to select the character with higher probability, because choosing the second highest might still lead us to meaningful text. It is more wise to **sample** characters from the probability distribution given by the network output.

This sampling can be done using `multinomial` function that implements so-called **multinomial distribution**. A function that implements this **soft** text generation is defined below:

但是，如果我们查看下一个字符的概率分布，可能是几个最高概率之间的差异并不大，例如 一个字符的概率为 0.2，另一个 - 0.19，等等。例如，当在序列“*play*”中查找下一个字符时，下一个字符同样可以是空格或 **e**（如 单词*玩家*）。

这使我们得出这样的结论：选择概率较高的字符并不总是“公平”，因为选择第二高的字符可能仍会引导我们找到有意义的文本。 从网络输出给出的概率分布中**采样**字符是更明智的做法。

可以使用实现所谓的**多项式分布**的`多项式`函数来完成此采样。 下面定义了一个实现这个**软**文本生成的函数：

In [15]:
def generate_soft(net,size=100,start='today ',temperature=1.0):
        chars = list(start)
        out, s = net(enc(chars).view(1,-1).to(device))
        for i in range(size):
            #nc = torch.argmax(out[0][-1])
            out_dist = out[0][-1].div(temperature).exp()
            nc = torch.multinomial(out_dist,1)[0]
            chars.append(vocab.itos[nc])
            out, s = net(nc.view(1,-1),s)
        return ''.join(chars)
    
for i in [0.3,0.8,1.0,1.3,1.8]:
    print(f"--- Temperature = {i}\n{generate_soft(net,size=300,start='Today ',temperature=i)}\n")

--- Temperature = 0.3
Today and to has a software to in the first the power the gold medal was of the first and succer to the company will a report the first the and the gain the company in the and a new a report a pack of the four the first the company of the such with the half to a security to the and a success the first she

--- Temperature = 0.8
Today drud out of the three-rent possiem that sales purssion has finminiaty women's from NAC Inc. (AP) -- Shimbon has weel with a may stelight first three flaw gold from their a scent, big study with a nighting sovicturner has slarh football at a hour of Angelage discression, into cubs, US year player sor

--- Temperature = 1.0
Today by compoy, said to hup the couns ay rrope iist\fill sinie-5-1- than he of a fightier Corp. the Vew, Mkli Unite Hold Austria on Tuesday resfare rextarted in the new has buy thisnillials thrust first capuration of the it larget expected the ir edulagy Airin Penny after Emonet Cuc Washieve an are Gurry

--- Temper

We have introduced one more parameter called **temperature**, which is used to indicate how hard we should stick to the highest probability. If temperature is 1.0, we do fair multinomial sampling, and when temperature goes to infinity - all probabilities become equal, and we randomly select next character. In the example below we can observe that the text becomes meaningless when we increase the temperature too much, and it resembles "cycled" hard-generated text when it becomes closer to 0. 